In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e4/sample_submission.csv
/kaggle/input/playground-series-s5e4/train.csv
/kaggle/input/playground-series-s5e4/test.csv


In [2]:
import pandas as pd
import numpy as np
df=pd.read_csv("/kaggle/input/playground-series-s5e4/train.csv")

In [3]:
df.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998
1,1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031


In [4]:
# Summary of missing values
missing_summary = df.isnull().sum().to_frame('Missing_Values')
missing_summary['Percent'] = 100 * missing_summary['Missing_Values'] / len(df)
missing_summary = missing_summary[missing_summary['Missing_Values'] > 0]

print("Missing Value Summary:")
print(missing_summary.sort_values(by='Percent', ascending=False))

Missing Value Summary:
                             Missing_Values    Percent
Guest_Popularity_percentage          146030  19.470667
Episode_Length_minutes                87093  11.612400
Number_of_Ads                             1   0.000133


In [5]:
from sklearn.impute import SimpleImputer

# Add missing indicators 
df['Episode_Length_missing'] = df['Episode_Length_minutes'].isnull().astype(int)
df['Guest_Popularity_missing'] = df['Guest_Popularity_percentage'].isnull().astype(int)

# Median imputation
median_imputer = SimpleImputer(strategy='median')
df[['Episode_Length_minutes', 'Guest_Popularity_percentage']] = median_imputer.fit_transform(
    df[['Episode_Length_minutes', 'Guest_Popularity_percentage']]
)

# One missing value in Number_of_Ads — simple median fill
df['Number_of_Ads'] = df['Number_of_Ads'].fillna(df['Number_of_Ads'].median())



In [6]:
df.isna().sum()

id                             0
Podcast_Name                   0
Episode_Title                  0
Episode_Length_minutes         0
Genre                          0
Host_Popularity_percentage     0
Publication_Day                0
Publication_Time               0
Guest_Popularity_percentage    0
Number_of_Ads                  0
Episode_Sentiment              0
Listening_Time_minutes         0
Episode_Length_missing         0
Guest_Popularity_missing       0
dtype: int64

In [7]:

import re

def process_episode_column(df, drop_original=True):
    """
    Extracts episode number from Episode_Title column and adds a new Episode_Number column.
    
    Parameters:
    - df: pandas DataFrame (train or test)
    - drop_original: whether to drop the original Episode_Title column (default: True)
    
    Returns:
    - df: DataFrame with Episode_Number added
    """
    # Extract numeric episode number
    def extract_episode_number(title):
        match = re.search(r'\d+', str(title))
        return int(match.group()) if match else None

    df['Episode_Number'] = df['Episode_Title'].apply(extract_episode_number)

    if drop_original:
        df.drop('Episode_Title', axis=1, inplace=True)

    return df

df = process_episode_column(df)

In [8]:
df.head()

,id,Podcast_Name,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes,Episode_Length_missing,Guest_Popularity_missing,Episode_Number
0,0,Mystery Matters,63.84,True Crime,74.81,Thursday,Night,53.58,0.0,Positive,31.41998,1,1,98
1,1,Joke Junction,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241,0,0,26
2,2,Study Sessions,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531,0,0,16
3,3,Digital Digest,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824,0,0,45
4,4,Mind & Body,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031,0,0,86


In [9]:
def analyze_categorical_columns(df, threshold=20):
    """
    Prints unique value count and examples for each object/categorical column in the DataFrame.

    Parameters:
    - df: pandas DataFrame
    - threshold: number of unique values below which all values are printed (default: 20)
    """
    categorical_cols = df.select_dtypes(include='object').columns.tolist()

    print(f"\n Found {len(categorical_cols)} categorical columns: {categorical_cols}")

    for col in categorical_cols:
        unique_vals = df[col].dropna().unique()
        print(f"\n🔹 Column: {col}")
        print(f"Unique Count: {len(unique_vals)}")

        if len(unique_vals) <= threshold:
            print(f"Unique Values: {unique_vals}")
        else:
            print(f"Sample Values: {unique_vals[:10]} ...")  # show sample if too many
            
analyze_categorical_columns(df)


 Found 5 categorical columns: ['Podcast_Name', 'Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']

🔹 Column: Podcast_Name
Unique Count: 48
Sample Values: ['Mystery Matters' 'Joke Junction' 'Study Sessions' 'Digital Digest'
 'Mind & Body' 'Fitness First' 'Criminal Minds' 'News Roundup'
 'Daily Digest' 'Music Matters'] ...

🔹 Column: Genre
Unique Count: 10
Unique Values: ['True Crime' 'Comedy' 'Education' 'Technology' 'Health' 'News' 'Music'
 'Sports' 'Business' 'Lifestyle']

🔹 Column: Publication_Day
Unique Count: 7
Unique Values: ['Thursday' 'Saturday' 'Tuesday' 'Monday' 'Sunday' 'Wednesday' 'Friday']

🔹 Column: Publication_Time
Unique Count: 4
Unique Values: ['Night' 'Afternoon' 'Evening' 'Morning']

🔹 Column: Episode_Sentiment
Unique Count: 3
Unique Values: ['Positive' 'Negative' 'Neutral']


In [10]:
def encode_categorical_features(df, categorical_cols):
    """
    Encodes categorical features based on provided list:
    - One-hot encodes 'Genre' and 'Publication_Time' if present.
    - Ordinal encodes 'Publication_Day' and 'Episode_Sentiment' if present.
    - Frequency encodes 'Podcast_Name' if present and drops it after encoding.
    Returns a transformed copy of the DataFrame.
    """
    df = df.copy()

    # One-hot encode if columns are present
    one_hot_cols = [col for col in ['Genre', 'Publication_Time'] if col in categorical_cols and col in df.columns]
    if one_hot_cols:
        df = pd.get_dummies(df, columns=one_hot_cols, drop_first=True)

    # Ordinal encode 'Publication_Day'
    if 'Publication_Day' in categorical_cols and 'Publication_Day' in df.columns:
        day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        df['Publication_Day'] = df['Publication_Day'].map({day: i for i, day in enumerate(day_order)})

    # Ordinal encode 'Episode_Sentiment'
    if 'Episode_Sentiment' in categorical_cols and 'Episode_Sentiment' in df.columns:
        sentiment_order = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
        df['Episode_Sentiment'] = df['Episode_Sentiment'].map(sentiment_order)

    # Frequency encode 'Podcast_Name'
    if 'Podcast_Name' in categorical_cols and 'Podcast_Name' in df.columns:
        podcast_freq = df['Podcast_Name'].value_counts(normalize=True)
        df['Podcast_Name_FE'] = df['Podcast_Name'].map(podcast_freq)
        df.drop('Podcast_Name', axis=1, inplace=True)

    return df
categorical_cols = df.select_dtypes(include='object').columns.tolist()
df = encode_categorical_features(df, categorical_cols)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 24 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           750000 non-null  int64  
 1   Episode_Length_minutes       750000 non-null  float64
 2   Host_Popularity_percentage   750000 non-null  float64
 3   Publication_Day              750000 non-null  int64  
 4   Guest_Popularity_percentage  750000 non-null  float64
 5   Number_of_Ads                750000 non-null  float64
 6   Episode_Sentiment            750000 non-null  int64  
 7   Listening_Time_minutes       750000 non-null  float64
 8   Episode_Length_missing       750000 non-null  int64  
 9   Guest_Popularity_missing     750000 non-null  int64  
 10  Episode_Number               750000 non-null  int64  
 11  Genre_Comedy                 750000 non-null  bool   
 12  Genre_Education              750000 non-null  bool   
 13 

In [12]:
df.shape

(750000, 24)

In [13]:
df.head()

,id,Episode_Length_minutes,Host_Popularity_percentage,Publication_Day,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes,Episode_Length_missing,Guest_Popularity_missing,...,Genre_Lifestyle,Genre_Music,Genre_News,Genre_Sports,Genre_Technology,Genre_True Crime,Publication_Time_Evening,Publication_Time_Morning,Publication_Time_Night,Podcast_Name_FE
0,0,63.84,74.81,3,53.58,0.0,2,31.41998,1,1,...,False,False,False,False,False,True,False,False,True,0.021336
1,1,119.80,66.95,5,75.95,2.0,0,88.01241,0,0,...,False,False,False,False,False,False,False,False,False,0.020099
2,2,73.90,69.97,1,8.97,0.0,0,44.92531,0,0,...,False,False,False,False,False,False,True,False,False,0.017404
3,3,67.17,57.22,0,78.70,2.0,2,46.27824,0,0,...,False,False,False,False,True,False,False,True,False,0.021561
4,4,110.51,80.07,0,58.68,3.0,1,75.61031,0,0,...,False,False,False,False,False,False,False,False,False,0.018185


In [14]:
df.drop(columns=['Episode_Length_missing', 'Guest_Popularity_missing'], inplace=True)


In [15]:
df.shape

(750000, 22)

In [16]:
from sklearn.model_selection import train_test_split, KFold

# Separate features and target
X = df.drop(columns=['Listening_Time_minutes', 'id'])
y = df['Listening_Time_minutes']

# Optional: quick validation split (not used in CV but good for debugging)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# K-Fold Cross Validation setup (we'll plug this into model evaluation later)
kfold = KFold(n_splits=5, shuffle=True, random_state=42)


In [17]:
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, make_scorer
# Define RMSE scoring
rmse_scorer = make_scorer(mean_squared_error, squared=False)

In [18]:
# Define the XGBoost model
xgb_model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    tree_method="hist",  # fast and efficient on CPU
    n_jobs=-1
)

In [19]:

# Parameters
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Track RMSEs
fold = 1
rmse_scores = []

for train_index, val_index in kf.split(X_train):
    X_tr, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
    
    # Model
    model = XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        tree_method="hist",
        n_jobs=-1
    )
    
    # Train with early stopping
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        early_stopping_rounds=50,
        verbose=50  # Shows training progress every 50 rounds
    )
    
    # Predict and score
    preds = model.predict(X_val)
    rmse = mean_squared_error(y_val, preds, squared=False)
    rmse_scores.append(rmse)
    
    print(f"\n✅ Fold {fold} RMSE: {rmse:.4f}\n")
    fold += 1

# Final RMSE summary
print("🔍 Cross-Validated RMSE Summary:")
print("Scores:", rmse_scores)
print(f"Mean RMSE: {np.mean(rmse_scores):.4f}")
print(f"Std Dev RMSE: {np.std(rmse_scores):.4f}")


[0]	validation_0-rmse:26.10799


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[50]	validation_0-rmse:13.45535
[100]	validation_0-rmse:13.10556
[150]	validation_0-rmse:13.06725
[200]	validation_0-rmse:13.04929
[250]	validation_0-rmse:13.03557
[300]	validation_0-rmse:13.02472
[350]	validation_0-rmse:13.01586
[400]	validation_0-rmse:13.00629
[450]	validation_0-rmse:12.99738
[499]	validation_0-rmse:12.99139

✅ Fold 1 RMSE: 12.9914

[0]	validation_0-rmse:26.15220


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[50]	validation_0-rmse:13.53472
[100]	validation_0-rmse:13.17870
[150]	validation_0-rmse:13.14306
[200]	validation_0-rmse:13.12423
[250]	validation_0-rmse:13.11053
[300]	validation_0-rmse:13.09813
[350]	validation_0-rmse:13.08710
[400]	validation_0-rmse:13.07816
[450]	validation_0-rmse:13.07069
[499]	validation_0-rmse:13.06335

✅ Fold 2 RMSE: 13.0633

[0]	validation_0-rmse:26.13943


...

[50]	validation_0-rmse:13.46416
[100]	validation_0-rmse:13.10382
[150]	validation_0-rmse:13.06536
[200]	validation_0-rmse:13.04817
[250]	validation_0-rmse:13.03441
[300]	validation_0-rmse:13.02420
[350]	validation_0-rmse:13.01421
[400]	validation_0-rmse:13.00554
[450]	validation_0-rmse:12.99865
[499]	validation_0-rmse:12.99306

✅ Fold 3 RMSE: 12.9931

[0]	validation_0-rmse:26.07094


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[50]	validation_0-rmse:13.52460
[100]	validation_0-rmse:13.18994
[150]	validation_0-rmse:13.15807
[200]	validation_0-rmse:13.14287
[250]	validation_0-rmse:13.13093
[300]	validation_0-rmse:13.12055
[350]	validation_0-rmse:13.11165
[400]	validation_0-rmse:13.10305
[450]	validation_0-rmse:13.09618
[499]	validation_0-rmse:13.08880

✅ Fold 4 RMSE: 13.0888

[0]	validation_0-rmse:26.11394


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[50]	validation_0-rmse:13.52275
[100]	validation_0-rmse:13.17126
[150]	validation_0-rmse:13.13550
[200]	validation_0-rmse:13.11962
[250]	validation_0-rmse:13.10704
[300]	validation_0-rmse:13.09522
[350]	validation_0-rmse:13.08525
[400]	validation_0-rmse:13.07617
[450]	validation_0-rmse:13.06879
[499]	validation_0-rmse:13.06146

✅ Fold 5 RMSE: 13.0615

🔍 Cross-Validated RMSE Summary:
Scores: [12.991387438704571, 13.06334939242185, 12.993060796511022, 13.088799913294379, 13.061463687400694]
Mean RMSE: 13.0396
Std Dev RMSE: 0.0399


In [20]:
test=pd.read_csv("/kaggle/input/playground-series-s5e4/test.csv")
test.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment
0,750000,Educational Nuggets,Episode 73,78.96,Education,38.11,Saturday,Evening,53.33,1.0,Neutral
1,750001,Sound Waves,Episode 23,27.87,Music,71.29,Sunday,Morning,NaN,0.0,Neutral
2,750002,Joke Junction,Episode 11,69.10,Comedy,67.89,Friday,Evening,97.51,0.0,Positive
3,750003,Comedy Corner,Episode 73,115.39,Comedy,23.40,Sunday,Morning,51.75,2.0,Positive
4,750004,Life Lessons,Episode 50,72.32,Lifestyle,58.10,Wednesday,Morning,11.30,2.0,Neutral


In [21]:
test = process_episode_column(test)
test.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,Podcast_Name,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Episode_Number
0,750000,Educational Nuggets,78.96,Education,38.11,Saturday,Evening,53.33,1.0,Neutral,73
1,750001,Sound Waves,27.87,Music,71.29,Sunday,Morning,NaN,0.0,Neutral,23
2,750002,Joke Junction,69.10,Comedy,67.89,Friday,Evening,97.51,0.0,Positive,11
3,750003,Comedy Corner,115.39,Comedy,23.40,Sunday,Morning,51.75,2.0,Positive,73
4,750004,Life Lessons,72.32,Lifestyle,58.10,Wednesday,Morning,11.30,2.0,Neutral,50


In [22]:
analyze_categorical_columns(test)


 Found 5 categorical columns: ['Podcast_Name', 'Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']

🔹 Column: Podcast_Name
Unique Count: 48
Sample Values: ['Educational Nuggets' 'Sound Waves' 'Joke Junction' 'Comedy Corner'
 'Life Lessons' 'Money Matters' 'Mystery Matters' 'Gadget Geek'
 'Home & Living' 'True Crime Stories'] ...

🔹 Column: Genre
Unique Count: 10
Unique Values: ['Education' 'Music' 'Comedy' 'Lifestyle' 'Business' 'True Crime'
 'Technology' 'News' 'Sports' 'Health']

🔹 Column: Publication_Day
Unique Count: 7
Unique Values: ['Saturday' 'Sunday' 'Friday' 'Wednesday' 'Tuesday' 'Thursday' 'Monday']

🔹 Column: Publication_Time
Unique Count: 4
Unique Values: ['Evening' 'Morning' 'Night' 'Afternoon']

🔹 Column: Episode_Sentiment
Unique Count: 3
Unique Values: ['Neutral' 'Positive' 'Negative']


In [23]:
categorical_cols = test.select_dtypes(include='object').columns.tolist()
test_encoded = encode_categorical_features(test, categorical_cols)
test_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 21 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           250000 non-null  int64  
 1   Episode_Length_minutes       221264 non-null  float64
 2   Host_Popularity_percentage   250000 non-null  float64
 3   Publication_Day              250000 non-null  int64  
 4   Guest_Popularity_percentage  201168 non-null  float64
 5   Number_of_Ads                250000 non-null  float64
 6   Episode_Sentiment            250000 non-null  int64  
 7   Episode_Number               250000 non-null  int64  
 8   Genre_Comedy                 250000 non-null  bool   
 9   Genre_Education              250000 non-null  bool   
 10  Genre_Health                 250000 non-null  bool   
 11  Genre_Lifestyle              250000 non-null  bool   
 12  Genre_Music                  250000 non-null  bool   
 13 

In [24]:

# Train final model on entire training set
final_model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    tree_method="hist",
    n_jobs=-1
)

final_model.fit(X_train, y_train, verbose=100)



XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=-1,
             num_parallel_tree=None, random_state=42, ...)

In [25]:
# Backup test IDs for submission
test_ids = test_encoded['id'].copy()
test_ids


0         750000
1         750001
2         750002
3         750003
4         750004
           ...  
249995    999995
249996    999996
249997    999997
249998    999998
249999    999999
Name: id, Length: 250000, dtype: int64

In [26]:
# Drop 'id' column before prediction
xtest = test_encoded.drop(columns=['id'])
xtest

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Episode_Length_minutes,Host_Popularity_percentage,Publication_Day,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Episode_Number,Genre_Comedy,Genre_Education,Genre_Health,Genre_Lifestyle,Genre_Music,Genre_News,Genre_Sports,Genre_Technology,Genre_True Crime,Publication_Time_Evening,Publication_Time_Morning,Publication_Time_Night,Podcast_Name_FE
0,78.96,38.11,5,53.33,1.0,1,73,False,True,False,False,False,False,False,False,False,True,False,False,0.015852
1,27.87,71.29,6,NaN,0.0,1,23,False,False,False,False,True,False,False,False,False,False,True,False,0.018756
2,69.10,67.89,4,97.51,0.0,2,11,True,False,False,False,False,False,False,False,False,True,False,False,0.020156
3,115.39,23.40,6,51.75,2.0,2,73,True,False,False,False,False,False,False,False,False,False,True,False,0.021032
4,72.32,58.10,2,11.30,2.0,1,50,False,False,False,True,False,False,False,False,False,False,True,False,0.019292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,21.05,65.77,5,96.40,3.0,0,100,False,False,True,False,False,False,False,False,False,True,False,False,0.018324
249996,85.50,41.47,5,30.52,2.0,0,85,True,False,False,False,False,False,False,False,False,False,False,True,0.020156
249997,12.11,25.92,3,73.69,1.0,1,63,True,False,False,False,False,False,False,False,False,True,False,False,0.020156
249998,113.46,43.47,4,93.59,3.0,2,46,False,False,False,False,False,False,False,False,False,False,False,True,0.017120


In [27]:

# Predict using your final trained model (assumes you've already trained it)
test_preds = final_model.predict(xtest)

# Create submission DataFrame
submission = pd.DataFrame({
    'id': test_ids,
    'listening_time': test_preds
})

# Save submission to CSV
submission.to_csv('submission.csv', index=False)

print("✅ Submission file 'submission.csv' created.")
submission.head()


✅ Submission file 'submission.csv' created.


,id,listening_time
0,750000,55.499969
1,750001,17.999048
2,750002,49.781666
3,750003,77.937363
4,750004,48.181587
